<a href="https://colab.research.google.com/github/Annieng184/Annieng184/blob/main/%5BProject%5D_Tiki_Web_Scraping_July_7th.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [183]:
# install selenium and other resources for crawling data
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/ma

In [184]:
import re
import time
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException

In [185]:
DRIVER = None

In [186]:
def initialize_driver():
    global DRIVER
    if DRIVER is None:
        print('Initiating driver...')
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument('-headless') 
        chrome_options.add_argument('-no-sandbox') 
        DRIVER = webdriver.Chrome('chromedriver',options=chrome_options)  # Create the new chrome browser with specific options
        print('Finished!')

In [187]:
# Global driver to use throughout the script
DRIVER = None

# Wrapper to close driver if its created
def close_driver():
    global DRIVER
    if DRIVER is not None:
        DRIVER.close()
    DRIVER = None

# Function to (re)start driver
def start_driver(force_restart=False):
    global DRIVER
    
    if force_restart:
        close_driver()
    
    # Setting up the driver
    options = webdriver.ChromeOptions()
    options.add_argument('-headless') # we don't want a chrome browser opens, so it will run in the background
    options.add_argument('-no-sandbox')
    options.add_argument('-disable-dev-shm-usage')

    DRIVER = webdriver.Chrome('chromedriver',options=options)

In [188]:
def close_driver():
    global DRIVER
    if not DRIVER is None:
        print("Quitting driver...")
        DRIVER.quit()
        print("Done!")

    DRIVER = None

In [189]:
close_driver()
initialize_driver()

Initiating driver...
Finished!


In [190]:
DRIVER

<selenium.webdriver.chrome.webdriver.WebDriver (session="2e74f7d085df997e1f6fcbdd95322f6c")>

In [191]:
import pandas as pd
import numpy as np

**PART 1: Input URL and find number of products within that site**

In [192]:
DRIVER.get('https://tiki.vn/dien-thoai-may-tinh-bang/c1789') #input URL

In [193]:
time.sleep(5) #wait for the website to load

In [194]:
products = DRIVER.find_elements(By.CLASS_NAME, 'product-item') #count elements
len(products)

48

In [195]:
each_product = products[0] #get 1 element out for example

In [196]:
each_product.get_attribute

<bound method WebElement.get_attribute of <selenium.webdriver.remote.webelement.WebElement (session="2e74f7d085df997e1f6fcbdd95322f6c", element="6d5501fd-6781-424b-ab42-2d8feaa48347")>>

In [197]:
each_product.get_attribute('outerHTML')

'<a class="product-item" href="//tka.tiki.vn/pixel/pixel?data=djAwMaaUEufulIH8mxk_7zplKNgn412wV0ydsggJvdEHzW-POs2991EOXJsoriY4cCO4t54Zye59L7XNl_LINZNsYdOTgYsVxJkaADksQRVyaOMPyk1GglWrjmZ7JPHV0qWeWZwCrh37_jwQcXQbYeC1mlK8ijyU55WvX7CGju6nW8AUWe54g6J2UmnwWszBOIw2mI-3vA_e9gI3zxnGdF9_aRjxTBEM0d06y4L53zVRYQlB3fn1burFJMxMwZKgC9vHOAVEnLAVNCHY5eGPyG5M2tUxArcznPZ6B1qHJKR2nwI7_4zyAEqoguQQoW1YOK2FO7SPiZ-uOEJdDK3eSXMfyZrZLQe7VwcIz7ym_8VyR76qspIvjrOoDCYp402jLWx6_yOiBIYThIVAsv-8EMH15XbXdGJPw_25f2fBKs4Wu3VvJcH7bGtYTJ-GXb-9YyNdmegR6BzGenyJ7mTrqmP6aRMQ1Hc877bZPgtQkPrO0r1fKAG9ldkWeMCLl4PuFVp1R9Hk9LlRe_bohSX8cZ7e0t4QyKuVkXz8bnw9ODbepCvCJUL-FPcg_G6s02Br91ngjc244--iTHKX-EbrrD72vtwY4V0GJYQe71oFc8FzVINwX7POHeIZopiTmpF4eKBTeb075EdTp5EO4XIa03uJ4aDqVmdnFdiLj03zwhZyxMIL_US9ZRbvkiMgN7vcNs7knDriSQTdEjGaXnLBYAi4-7O-uEJjNjSEiLg56wAiMKUeFa8Nek1uSwbaouU1MR8OQYSMv4pdgHchzGmDUcs5hDrHC6pYktjjs6nZdmDpICPE7ZyQy_oTbypVq2KXP1RnO4avWy8mxH0QHjnhqdOdIO9eQXuIXMJ3Hx91DlOb1SnJuyteJo0KoIsW1kvLWOkVDrsTfHtQAhuYCV8G6_HGgP5GuAaQbQIZ2Q8aAi

**PART 2: Find 4 elements: name, price, product_url, image**

In [198]:
#name
name = each_product.find_element(By.TAG_NAME, 'h3')
print(name.text)

Điện thoại Samsung Galaxy M23 5G (6GB/128GB) - Hàng chính hãng


In [199]:
#price
price = each_product.find_element(By.CLASS_NAME, 'price-discount__price')
print(price.get_attribute('innerHTML'))

4.390.000 ₫


In [200]:
#get product url
url = each_product.get_attribute('href')
print(url)

https://tka.tiki.vn/pixel/pixel?data=djAwMaaUEufulIH8mxk_7zplKNgn412wV0ydsggJvdEHzW-POs2991EOXJsoriY4cCO4t54Zye59L7XNl_LINZNsYdOTgYsVxJkaADksQRVyaOMPyk1GglWrjmZ7JPHV0qWeWZwCrh37_jwQcXQbYeC1mlK8ijyU55WvX7CGju6nW8AUWe54g6J2UmnwWszBOIw2mI-3vA_e9gI3zxnGdF9_aRjxTBEM0d06y4L53zVRYQlB3fn1burFJMxMwZKgC9vHOAVEnLAVNCHY5eGPyG5M2tUxArcznPZ6B1qHJKR2nwI7_4zyAEqoguQQoW1YOK2FO7SPiZ-uOEJdDK3eSXMfyZrZLQe7VwcIz7ym_8VyR76qspIvjrOoDCYp402jLWx6_yOiBIYThIVAsv-8EMH15XbXdGJPw_25f2fBKs4Wu3VvJcH7bGtYTJ-GXb-9YyNdmegR6BzGenyJ7mTrqmP6aRMQ1Hc877bZPgtQkPrO0r1fKAG9ldkWeMCLl4PuFVp1R9Hk9LlRe_bohSX8cZ7e0t4QyKuVkXz8bnw9ODbepCvCJUL-FPcg_G6s02Br91ngjc244--iTHKX-EbrrD72vtwY4V0GJYQe71oFc8FzVINwX7POHeIZopiTmpF4eKBTeb075EdTp5EO4XIa03uJ4aDqVmdnFdiLj03zwhZyxMIL_US9ZRbvkiMgN7vcNs7knDriSQTdEjGaXnLBYAi4-7O-uEJjNjSEiLg56wAiMKUeFa8Nek1uSwbaouU1MR8OQYSMv4pdgHchzGmDUcs5hDrHC6pYktjjs6nZdmDpICPE7ZyQy_oTbypVq2KXP1RnO4avWy8mxH0QHjnhqdOdIO9eQXuIXMJ3Hx91DlOb1SnJuyteJo0KoIsW1kvLWOkVDrsTfHtQAhuYCV8G6_HGgP5GuAaQbQIZ2Q8aAim5PQwZvPyHWz7D9TKCD0M-GTu

In [201]:
#image
image = each_product.find_element(By.TAG_NAME, 'source')
print(image.get_attribute('srcset'))

https://salt.tikicdn.com/cache/200x200/ts/product/8d/1b/e9/6c4b0e5309a8f34c8db4910e35b9f51d.jpg.webp


In [202]:
#get number of sold items
sales = each_product.find_element(By.CLASS_NAME, 'styles__StyledQtySold-sc-732h27-2').get_attribute('innerHTML').split(' ')[2].strip(' ')
print(sales)

1000+


In [203]:
#get discount of items
discount = each_product.find_element(By.CLASS_NAME, 'price-discount__discount').get_attribute('innerHTML').strip('%,-')
print(discount)

36


In [204]:
#check if item have Tiki Now
# tiki_now = each_product.find_element(By.CLASS_NAME, 'badge-service').get_attribute('innerHTML')
print(tiki_now)
def tiki_now_check(tiki_now):
  if tiki_now is None:
    return False
  else:
    return True
print(tiki_now_check(tiki_now))

<div class="item"><div><img width="56" height="16" src="https://salt.tikicdn.com/ts/upload/9f/32/dd/8a8d39d4453399569dfb3e80fe01de75.png"></div><span style="color: rgb(0, 171, 86);">GIAO SÁNG MAI</span></div>
True


In [205]:
#create a function that aggregates the above information
def scrap_item(item):
  try:
    name = item.find_element(By.TAG_NAME, 'h3').get_attribute('innerHTML')
  except:
    name = ''
  try:
    price = item.find_element(By.CLASS_NAME, 'price-discount__price').get_attribute('innerHTML')
  except:
    price = ''
  try:
    url = item.get_attribute('href')
  except:
    url = ''
  try:
    image = item.find_element(By.TAG_NAME, 'source').get_attribute('srcset')
  except:
    image = ''
  try:
    sales = item.find_element(By.CLASS_NAME, 'styles__StyledQtySold-sc-732h27-2').get_attribute('innerHTML').split(' ')[2].strip('+')
  except:
    sales = ''
  try:
    discount = item.find_element(By.CLASS_NAME, 'price-discount__discount').get_attribute('innerHTML').strip('%,-')
  except:
    discount = ''
  try:
    tiki_now = tiki_now_check(item.find_element(By.CLASS_NAME, 'badge-service').get_attribute('innerHTML'))
  except:
    tiki_now = False
  try:
    free_ship = free_ship_check(item.find_element(By.CLASS_NAME, 'thumbnail').get_attribute('https://salt.tikicdn.com/ts/upload/dc/0d/49/3251737db2de83b74eba8a9ad6d03338.png'))
  except:
    free_ship = False

  info = {
      'name' : name,
      'price' : price,
      'url': url,
      'image' : image,
      'sales' : sales,
      'discount' : discount,
      'tiki_now' : tiki_now,
  }
  return info

In [206]:
extracted_items = []


In [207]:
from selenium.common.exceptions import StaleElementReferenceException

In [208]:
for item in products:
  extracted_items.append(scrap_item(item))

In [209]:
print(extracted_items)

[{'name': 'Điện thoại Samsung Galaxy M23 5G (6GB/128GB) - Hàng chính hãng', 'price': '4.390.000 ₫', 'url': 'https://tka.tiki.vn/pixel/pixel?data=djAwMaaUEufulIH8mxk_7zplKNgn412wV0ydsggJvdEHzW-POs2991EOXJsoriY4cCO4t54Zye59L7XNl_LINZNsYdOTgYsVxJkaADksQRVyaOMPyk1GglWrjmZ7JPHV0qWeWZwCrh37_jwQcXQbYeC1mlK8ijyU55WvX7CGju6nW8AUWe54g6J2UmnwWszBOIw2mI-3vA_e9gI3zxnGdF9_aRjxTBEM0d06y4L53zVRYQlB3fn1burFJMxMwZKgC9vHOAVEnLAVNCHY5eGPyG5M2tUxArcznPZ6B1qHJKR2nwI7_4zyAEqoguQQoW1YOK2FO7SPiZ-uOEJdDK3eSXMfyZrZLQe7VwcIz7ym_8VyR76qspIvjrOoDCYp402jLWx6_yOiBIYThIVAsv-8EMH15XbXdGJPw_25f2fBKs4Wu3VvJcH7bGtYTJ-GXb-9YyNdmegR6BzGenyJ7mTrqmP6aRMQ1Hc877bZPgtQkPrO0r1fKAG9ldkWeMCLl4PuFVp1R9Hk9LlRe_bohSX8cZ7e0t4QyKuVkXz8bnw9ODbepCvCJUL-FPcg_G6s02Br91ngjc244--iTHKX-EbrrD72vtwY4V0GJYQe71oFc8FzVINwX7POHeIZopiTmpF4eKBTeb075EdTp5EO4XIa03uJ4aDqVmdnFdiLj03zwhZyxMIL_US9ZRbvkiMgN7vcNs7knDriSQTdEjGaXnLBYAi4-7O-uEJjNjSEiLg56wAiMKUeFa8Nek1uSwbaouU1MR8OQYSMv4pdgHchzGmDUcs5hDrHC6pYktjjs6nZdmDpICPE7ZyQy_oTbypVq2KXP1RnO4avWy8mxH0QHjnhqdO

**PART 3: Save results as CSV file**

In [210]:
df = pd.DataFrame(extracted_items, columns=['name', 'price', 'url', 'image', 'sales', 'discount', 'tiki_now'])

In [211]:
df.index = np.arange(1, len(df) + 1)

In [212]:
df


,name,price,url,image,sales,discount,tiki_now
1,Điện thoại Samsung Galaxy M23 5G (6GB/128GB) -...,4.390.000 ₫,https://tka.tiki.vn/pixel/pixel?data=djAwMaaUE...,https://salt.tikicdn.com/cache/200x200/ts/prod...,1000,36,False
2,Điện thoại Samsung Galaxy A52s 5G (8GB/128GB) ...,8.490.000 ₫,https://tka.tiki.vn/pixel/pixel?data=djAwMUkg4...,https://salt.tikicdn.com/cache/200x200/ts/prod...,1000,6,False
3,Điện thoại Samsung M33 5G (8Gb/128GB) - Hàng c...,4.790.000 ₫,https://tka.tiki.vn/pixel/pixel?data=djAwMSfqE...,https://salt.tikicdn.com/cache/200x200/ts/prod...,581,40,False
4,Máy Đọc Sách All New Kindle 2019 (10th) – Hàng...,2.245.000 ₫,https://tka.tiki.vn/pixel/pixel?data=djAwMRLX0...,https://salt.tikicdn.com/cache/200x200/ts/prod...,441,2,False
5,Điện thoại Samsung Galaxy M51 - Hàng Chính Hãng,7.690.000 ₫,https://tiki.vn/dien-thoai-samsung-galaxy-m51-...,https://salt.tikicdn.com/cache/200x200/ts/prod...,1000,19,False
6,Máy tính bảng Samsung Galaxy Tab A7 Lite LTE S...,3.490.000 ₫,https://tiki.vn/may-tinh-bang-samsung-galaxy-t...,https://salt.tikicdn.com/cache/200x200/ts/prod...,1000,13,False
7,Điện thoại Xiaomi Redmi 9A (2GB/32GB) - Hàng c...,2.109.000 ₫,https://tiki.vn/dien-thoai-xiaomi-redmi-9a-2gb...,https://salt.tikicdn.com/cache/200x200/ts/prod...,53,15,False
8,Máy tính bảng Samsung Galaxy Tab A8 (4GB/64GB)...,5.890.000 ₫,https://tiki.vn/may-tinh-bang-samsung-galaxy-t...,https://salt.tikicdn.com/cache/200x200/ts/prod...,87,31,False
9,Điện Thoại Samsung Galaxy M32 (8GB/128GB) - Hà...,4.490.000 ₫,https://tiki.vn/dien-thoai-samsung-galaxy-m32-...,https://salt.tikicdn.com/cache/200x200/ts/prod...,371,25,True
10,Điện thoại Samsung Galaxy A03s (4GB/64GB) - Hà...,3.090.000 ₫,https://tiki.vn/dien-thoai-samsung-galaxy-a03s...,https://salt.tikicdn.com/cache/200x200/ts/prod...,1000,,True


In [213]:
df.to_csv('tiki.scraped.csv', index = True)